In [2]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

In [ ]:
train = pd.read_csv('data/train.csv', chunksize = 1000)
train_dict = {}
for chunk in train
